In [1]:
from IPython import display
from mxnet import nd,autograd
import random

In [2]:
num_inputs = 3
num_examples = 1000
true_w = [3,-1,7.3]
true_b = 5.7
features = nd.random.normal(scale=1,shape=(num_examples,num_inputs))
labels = true_w[0] * features[:,0] + true_w[1] * features[:,1] + true_w[2] * features[:,2] + true_b
labels += nd.random.normal(scale=0.1,shape=labels.shape)


In [3]:
def data_iter(batch_size,features,labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)   #打乱索引
    for i in range(0,num_examples,batch_size):
        j=nd.array(indices[i:min(i+batch_size,num_examples)])
        yield features.take(j),labels.take(j)

In [30]:
batch_size = 10
for x,y in data_iter(batch_size,features,labels):
    print(x,y)
    break    #测试一次


[[ 0.03820492 -1.0871673   0.17046909]
 [-1.0010443   1.624257   -0.9838195 ]
 [ 0.75896156  0.19741666  0.78231204]
 [ 0.62100744 -1.8929878   1.3092364 ]
 [ 1.1983087  -0.7036396  -1.5554514 ]
 [-1.3360573  -0.27023232  2.2202117 ]
 [-0.20937306  1.058954    1.4487612 ]
 [ 0.0591518  -0.49726588  0.1703758 ]
 [-0.19660912  1.1343161  -1.110928  ]
 [ 0.33467403 -0.17202017  0.88323224]]
<NDArray 10x3 @cpu(0)> 
[ 8.137586  -6.026309  13.3549795 19.079487  -1.3290368 18.15236
 14.744698   7.5922956 -4.2614546 13.438318 ]
<NDArray 10 @cpu(0)>


In [31]:
w = nd.random.normal(scale=0.01,shape=(num_inputs,1))
b = nd.ones(shape=(1))
w.attach_grad()
b.attach_grad()

In [32]:
def linreg(x,w,b):
    return nd.dot(x,w)+b

In [33]:
def squares_loss(y_hat,y):
    return (y_hat-y.reshape(y_hat.shape)) **2 / 2

In [34]:
def sgd(params,lr,batch_size):
    for param in params:
        param[:] = param - lr*param.grad/batch_size

In [35]:
lr=0.01
num_epochs=20
for epoch in range(num_epochs):
    for x,y in data_iter(batch_size, features, labels):
        with autograd.record():
            l=squares_loss(linreg(x,w,b),y)
        l.backward()
        sgd([w,b],lr,batch_size)
    train_l=squares_loss(linreg(features,w,b),labels)
    print('epoch %d,loss %f'%(epoch+1,train_l.mean().asnumpy()))

epoch 1,loss 5.983449
epoch 2,loss 0.900064
epoch 3,loss 0.141336
epoch 4,loss 0.025775
epoch 5,loss 0.008026
epoch 6,loss 0.005290
epoch 7,loss 0.004856
epoch 8,loss 0.004798
epoch 9,loss 0.004785
epoch 10,loss 0.004784
epoch 11,loss 0.004784
epoch 12,loss 0.004783
epoch 13,loss 0.004783
epoch 14,loss 0.004783
epoch 15,loss 0.004783
epoch 16,loss 0.004784
epoch 17,loss 0.004783
epoch 18,loss 0.004783
epoch 19,loss 0.004783
epoch 20,loss 0.004783


In [26]:
true_w,w

([3, -1, 7.3], 
 [[ 0.03132214]
  [-0.00435899]
  [-0.00492951]]
 <NDArray 3x1 @cpu(0)>)

In [11]:
true_b,b

(5.7, 
 [1.]
 <NDArray 1 @cpu(0)>)